# Group ARD
Starting to work on this Nov 19, 2015. Given $N$ samples with $M$ dimensions each (selected from $\mathcal{N}(0,1)$), $N$ target values $Y$ (with Gaussian noise added, $\sigma^2$ chosen), and a set of weight vectors $w$ (from an exponential distribution, $\lambda$ chosen, with a certain proportion of the weights (Zeros) set to 0), preform Group ARD and attempt to recover the weights. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

%matplotlib inline

In [ ]:
# Generate random design matrix. 
N = 100000     # sample size
groups = 10
in_group = 25
M = groups * in_group      # dimensions
Zeros = 0.6 # Proportion of zero weights (0 to 1)
noise = 1.0  # sigma^2 for Gaussian noise added to Y

M_zero = int(Zeros*groups)
X = np.random.normal(0, 1, (N,M))/M

In [ ]:
# Generate weights
w = np.random.laplace(0, 10, (groups,in_group))
perm = np.random.permutation(groups)
w[perm[:M_zero]] = 0
w = np.ravel(w)

# Plotting
f ,ax = plt.subplots(1,2, figsize=(16,5))
ax[0].plot(w)
ax[1].hist(w, bins=np.arange(min(w), max(w) + 1, 2))
plt.show()

In [ ]:
# Generate Y
Y_noiseless = np.dot(X,w)
Y = Y_noiseless + np.random.normal(0,noise, len(Y_noiseless))  # Add Gaussian Noise

# Plotting
f ,ax = plt.subplots(1,1, figsize=(16,5))
ax.plot(Y, color= 'blue')
ax.plot(Y_noiseless, color = 'green')
plt.show()

In [ ]:
# Group ARD Initialization
alpha_max = 10
max_iterations = 1000

a = np.repeat(1.0, M) # alphas
b = 1.0               # Beta = 1/sig^2 

X1 = X

deletions = []
old_alphas = [a]

for ard_iteration in range(max_iterations):
    
    # Sigma = (b*XTX + A)^-1
    Sigma = np.linalg.inv(b*np.dot(X1.T,X1) + np.diag(a))

    # mu = b*Sigma*X.T*Y
    mu = b*np.dot(np.dot(Sigma, X1.T), Y)
    
    gamma = 1.0 - a*np.diag(Sigma)
    group_gamma = np.array([np.sum(gamma[i:i+in_group]) for i in range(0, len(gamma), in_group)])
    mu_squared = mu**2
    group_mu = np.array([np.sum(mu_squared[i:i+in_group]) for i in range(0, len(mu_squared), in_group)])
    a_new = group_gamma/group_mu
    
    error = np.sum((Y - np.dot(X1, mu))**2)
    b_new = (N - np.sum(gamma))/error
    
    a = [alpha for alpha in a_new for k in range(in_group)]
    b = b_new
    
    print("\nIteration: ", ard_iteration, " beta = ", b, " Squared-Error = ", error)  
        
    over = [i for i in range(len(a))if a[i] > alpha_max]
    if over:
        print("Deletions: ", len(over))
        deletions = [over] + deletions
        X1 = np.delete(X1,over,axis=1)
        a = np.delete(a,over)
    else:
        a_converge = np.sum((a - np.array(old_alphas[-1]))**2)
        print("Alpha distance = ", a_converge, "   Max alpha = ", np.max(a))
        if a_converge < .00001:
            break
    old_alphas.append(a)
           

# Recover mu
for i in deletions:
    for j in i:
        a = np.insert(a,j,-1)
        mu = np.insert(mu,j,0)

df = pd.DataFrame(list(zip(a, mu, w, mu - w)), columns = ['alpha', 'mu', 'w', 'error'])
print("\n", df)
print("\nDeletions:", np.sum([len(i) for i in deletions])/in_group, "out of", M_zero)#, "\n", deletions)

In [ ]:
print(np.shape(X1), np.shape(Y), len(w), len(a), type(mu), type(w))

In [ ]:
# Plotting old weights + new weights, errors
f ,ax = plt.subplots(1,2, figsize=(16,5))
ax[0].plot(mu, color = 'green', label = r'$\mu$')
ax[0].plot(w, color = 'blue', label = r'$w$')
ax[1].plot(w - mu)

ax[0].legend(loc = "upper left", fontsize = 16)
ax[0].set_title(r'Original and Recovered Weights')
ax[0].set_xlabel(r'Weight #')
ax[0].set_ylabel(r'Weight Value')

ax[1].set_title(r'Original - Recovered Weights')
ax[1].set_xlabel(r'Weight #')
ax[1].set_ylabel(r'Error')

plt.show()